In [1]:
import pysal as ps
import pandas as pd
import numpy as np

In [2]:
def df2dbf(df, dbf_path, my_specs=None):
    '''
    Convert a pandas.DataFrame into a dbf.
    __author__  = "Dani Arribas-Bel <darribas@asu.edu> "
    ...
    Arguments
    ---------
    df          : DataFrame
                  Pandas dataframe object to be entirely written out to a dbf
    dbf_path    : str
                  Path to the output dbf. It is also returned by the function
    my_specs    : list
                  List with the field_specs to use for each column.
                  Defaults to None and applies the following scheme:
                    * int: ('N', 14, 0)
                    * float: ('N', 14, 14)
                    * str: ('C', 14, 0)
    '''
    if my_specs:
        specs = my_specs
    else:
        type2spec = {int: ('N', 20, 0),
                     np.int64: ('N', 20, 0),
                     float: ('N', 36, 15),
                     np.float64: ('N', 36, 15),
                     str: ('C', 14, 0)
                     }
        types = [type(df[i].iloc[0]) for i in df.columns]
        specs = [type2spec[t] for t in types]
    db = ps.open(dbf_path, 'w')
    db.header = list(df.columns)
    db.field_spec = specs
    for i, row in df.T.iteritems():
        db.write(row)
    db.close()
    return dbf_path


def dbf2df(dbf_path, index=None, cols=False, incl_index=False):
    '''
    Read a dbf file as a pandas.DataFrame, optionally selecting the index
    variable and which columns are to be loaded.
    __author__  = "Dani Arribas-Bel <darribas@asu.edu> "
    ...
    Arguments
    ---------
    dbf_path    : str
                  Path to the DBF file to be read
    index       : str
                  Name of the column to be used as the index of the DataFrame
    cols        : list
                  List with the names of the columns to be read into the
                  DataFrame. Defaults to False, which reads the whole dbf
    incl_index  : Boolean
                  If True index is included in the DataFrame as a
                  column too. Defaults to False
    Returns
    -------
    df          : DataFrame
                  pandas.DataFrame object created
    '''
    db = ps.open(dbf_path)
    if cols:
        if incl_index:
            cols.append(index)
        vars_to_read = cols
    else:
        vars_to_read = db.header
    data = dict([(var, db.by_col(var)) for var in vars_to_read])
    if index:
        index = db.by_col(index)
        db.close()
        return pd.DataFrame(data, index=index)
    else:
        db.close()
        return pd.DataFrame(data)

def SummarizeHAZUS(InputDBF,OutputDBF):
    df = dbf2df(InputDBF)
    df["CensusBloc"]=df["CensusBloc"].astype(str)
    df.set_index("CensusBloc",inplace=True)

    bldgbyblock = {}
    contbyblock = {}
    countbyblock= {}

    for i in range(len(df.index.unique())):
        censusblock = df.index.unique()[i]
        bldgloss = df.ix[censusblock].BldgLoss.sum()
        contloss = df.ix[censusblock].ContLoss.sum()
        bldcount = len(df.BldgLoss[df.index == censusblock])
        bldgbyblock[censusblock] = bldgloss
        contbyblock[censusblock] = contloss
        countbyblock[censusblock]= bldcount

    df_cont = pd.Series(contbyblock, name='ContLoss')
    df_cont.index.name = 'CensusBlock'
    df_bldg = pd.Series(bldgbyblock, name='BldgLoss')
    df_bldg.index.name = 'CensusBlock'
    df_count = pd.Series(countbyblock, name='BldCount')
    df_count.index.name = 'CensusBlock'

    losses = pd.concat([df_bldg,df_cont,df_count],axis=1)
    losses.reset_index(inplace=True)
    losses['CensusBlock'] = losses['CensusBlock'].astype(str)

    my_specs = [('C', 17, 17), ('N', 9, 3), ('N', 9, 3),('N', 3, 0)]
    df2dbf(losses,OutputDBF,my_specs=my_specs)

In [3]:
indbf = "/media/sf_Shared_Linux/test.dbf"
outdbf = "/media/sf_Shared_Linux/out.dbf"

SummarizeHAZUS(indbf,outdbf)